In [ ]:
#Pkg.update()
#Pkg.add("Distributions")
#Pkg.add("LightGraphs")
#Pkg.add("Cubature")
#include("PayloadGraph.jl")
#include("IM.jl")
#include("SIS.jl")
@everywhere using SIS,IM,PayloadGraph,PyPlot, Epidemics
import LightGraphs

## Set up Model

In [4]:

@everywhere begin

k = 4
#y_n = 0.1
c_r = 0.18
N = 5000 #1000
n_n = 2000#400#y_n*N
beta = 4.0/(c_r*n_n)
alpha = (N*beta)/n_n
println(N, ' ' ,alpha, ' ',beta)

im = InfectionModel(x -> 1 + alpha*x , x -> 1 + beta);
imk = InfectionModel(x -> 1 + beta + get_s_eff(x,alpha,beta,k) , x -> 1 + beta);

y_n, y_minus, y_plus, y_p,critical_determinant = get_parameters(N,alpha,beta)
end

5000 0.02777777777777778 0.011111111111111112
y_n = 0.39999999999999997, y_- = 0.018892297237251676, y_+ = 0.3811077027627483, y_p = 0.4172556098240043, critical determinant = 0.18000000000000002
	From worker 3:	5000 0.02777777777777778 0.011111111111111112
	From worker 2:	5000 0.02777777777777778 0.011111111111111112
'n_n = 1999.9999999999998	From worker 2:	y_n = 0.39999999999999997, y_- = 0.018892297237251676, y_+ = 0.3811077027627483, y_p = 0.4172556098240043, critical determinant = 0.18000000000000002

	From worker 2:	'n_n = 1999.9999999999998
	From worker 3:	y_n = 0.39999999999999997, y_- = 0.018892297237251676, y_+ = 0.3811077027627483, y_p = 0.4172556098240043, critical determinant = 0.18000000000000002
	From worker 3:	'n_n = 1999.9999999999998


Notes on P_fix:

- P_fix follows a logistical shape with the point of maximum slope = y_n.
- given $n_n$, it is independent of $N$. The larger $n_n$, the sharper the transition.
- the smaller $c_r < 1$, the sharper the transition at $y_n$. The larger $c_r < 1$, the shallower the transition
- For $c_r > 1$, the growth is linear and reaches $1$ at about $y = y_p$


# Plot Model

In [ ]:
function plot_schematic(n_n,c_r,N,k=N-1,plot_k=false)
    beta = 4.0/(c_r*n_n)
    alpha = (N*beta)/n_n
    println(N,alpha,beta)

    y_n, y_minus,y_plus,y_p,critical_determinant = get_parameters(N,alpha,beta)
    f(y) = alpha.*y.^2
    s(y) = f(y)./y - beta
    get_y_eff(y,k) = y.*(1 + (1-y)./(y.*k))
    get_s_eff(y,alpha,beta,k) = alpha*get_y_eff(y,k) - beta


    y_range = collect(0:y_p/1000:1.9*y_p)
    figure(1)
    plot(y_range,1.0./abs(N*s(y_range)),"-r",label=L"$\frac{1}{N|s(y)|}$")
    if plot_k
        plot(y_range,1.0./abs(N*get_s_eff(y_range,alpha,beta,k)),"-b",label=L"$\frac{1}{N|s(y_{eff})|}$")
    end
    plot(y_range,y_range,"-k",label=L"$y$")
    axvline(y_n,linestyle="--",label=L"$y_n$")
    axvline(y_p,linestyle="-",label=L"$y_p$")
    if y_minus > 0
        axvline(y_minus,linestyle="-.",label=L"$y_1$")
        axvline(y_plus,linestyle="-.",label=L"$y_2$")
    end
    ylim([0,1.9*y_p])
    legend(prop=Dict{Any,Any}("size"=>15),loc="upper right")
    xlabel(L"$y$",size=20)
    if plot_k
        #figure(1)
        #plot(y_range,get_s_eff(y_range,alpha,beta,k),"-b",label=L"$s_{eff}(y)$")
        #plot(y_range,s(y_range),"-r",label=L"$s(y)$")
        legend(prop=Dict{Any,Any}("size"=>20),loc="upper right")
        xlabel(L"$y$",size=20)
    end
end



In [ ]:
pygui(true)
close("all")
dx = 2*n_n/N/50
x = collect(dx:dx:2*n_n/N)
y = IM.P_fix(im,N,x)
yk = IM.P_fix(imk,N,x)
figure(2,figsize=(8,5))
plot(x,y,"-r",label=L"P_{fix}(y)")
plot(x,yk,"-b",label=L"P_{fix}(y_{eff})")
plot(x,x,"--k")
xlim([0,2*n_n/N])
y_n, y_minus, y_plus, y_p,critical_determinant = get_parameters(N,alpha,beta)
axvline(y_n,linestyle="--",color="b",label=L"y_n")
axvline(y_minus,linestyle="-.",color="r",label=L"y_1")
axvline(y_plus,linestyle="-.",color="r",label=L"y_2")
axvline(y_p,linestyle="-",color="b",label=L"y_p")
xlabel(L"y")
ylabel(L"P_{fix}(y)")
legend(loc="upper left")
title(latexstring("\$y_n = $(n_n/N), c_r = $c_r, N = $N\$"))
#savefig("p_fix_y_n = $(n_n/N), c_r = $c_r, N = $N.png")

figure(1,figsize=(8,5))
plot_schematic(n_n,c_r,N,k,true)
title(latexstring("\$y_n = $(n_n/N), c_r = $c_r, N = $N\$"))

figure(3)
plot(x,get_s_eff(x,alpha,beta,k),"-b",label=L"$s_{eff}(y)$")
plot(x,s(x,alpha,beta),"-r",label=L"$s(y)$")
grid()


## Get Data

In [ ]:
num_trials = 10000
num_trials_mixed = 100000
fixation_threshold = 2*y_n
regular=true

sizes_mixedk,num_fixed_mixedk,lengthk,runs_mixedk = 
run_epidemics(N,num_trials_mixed,im,(N,im)
    -> run_epidemic_well_mixed(N,imk,fixation_threshold))

sizes_mixed,num_fixed_mixed,length,runs_mixed = 
run_epidemics(N,num_trials_mixed,im,(N,im)
    -> run_epidemic_well_mixed(N,im,fixation_threshold))

# sizes_full,num_fixed_full,_,runs_full = 
#     run_epidemics(N,num_trials,im, (N,im)
#     -> run_epidemic_graph(N,N-1,im,regular,1.0))

@time sizes,num_fixed,_,runs = 
    run_epidemics(N,num_trials,im, (N,im)
    -> run_epidemic_graph(N,k,im,regular,fixation_threshold))


#sizes,num_fixed,length,runs = run_epidemics(N,num_trials,im, (N,im) -> run_epidemic_graph(N,k,im,regular))


allruns_mixedk = reduce(vcat,runs_mixedk)
allruns_mixed = reduce(vcat,runs_mixed)
#allruns_full = reduce(vcat,runs_full)
allruns = reduce(vcat,runs)


In [ ]:
maximum(allruns)

## P_fix

In [ ]:
println(num_fixed/num_trials," ", IM.P_fix(imk,N,1/N))

# println(num_fixed_full/num_trials," ",IM.P_fix(im,N,1/N))
# println(num_fixed_fullt/num_trials," ",IM.P_fix(im,N,1/N))


println(num_fixed_mixed/num_trials_mixed," ",IM.P_fix(im,N,1/N))

println(num_fixed_mixedk/num_trials_mixed," ",IM.P_fix(imk,N,1/N))


## Distribution vs. Time

In [ ]:
N = 20
time_arr = zeros(maximum([size(run) for run in runs_mixed])[1],N+1)
for run in runs_mixed
    for (i,val) in enumerate(run[1:end-1])
        time_arr[i,1+round(val)] += 1
    end
    T = size(run)[1]
    if run[T] >= 20
        time_arr[T:end,end] += 1
    elseif run[T] == 0
        time_arr[T:end,1] += 1
    else
        println("Problem")
        println(run[T])
    end
end

for i in 1:size(time_arr)[1]
    time_arr[i,:] /= sum(time_arr[i,:])
end

In [ ]:
using Interact

In [ ]:
f = figure()
@manipulate for i = 1:size(time_arr)[1]
    withfig(f) do
        PyPlot.semilogy(1:20,time_arr[i,:]'+1e-6)
        ylim([1e-7,1])
    end
end

In [ ]:
PyPlot.surf(log10(time_arr[:,2:end-1]+1e-6),rstride=20,cstride=2,alpha=0.4)
PyPlot.gca()[:view_init](elev=30,azim=210)
gca()[:set_xlabel](L"$t$")
gca()[:set_ylabel](L"$x$")
gca()[:set_zlabel](L"$P(x)$")
figure()
plot(time_arr[:,end])
xlabel(L"$t$")
ylabel(L"$P_{fix}(t)$")
figure()
plot(time_arr[:,1])
xlabel(L"$t$")
ylabel(L"$P_{extinct}(t)$")

## General Distribution averaged over time

In [ ]:
figure()
showlog = true
log_x = false
if log_x
    bins = linspace(log10(0.1),log10(n_n),20)
else
    bins = linspace(0,n_n,n_n)
end    
PyPlot.plt[:hist](allruns,bins=bins,alpha=0.2,normed=true,log=showlog,label="sparse")
PyPlot.plt[:hist](allruns_mixed,bins=bins,alpha=0.3,normed=true,log=showlog,label="mixed")
PyPlot.plt[:hist](allruns_mixedk,bins=bins,alpha=0.3,normed=true,log=showlog,label="mixed-k")
if log_x
    gca()[:set_xscale]("log")
end
#PyPlot.plt[:hist](allruns_full,bins=linspace(0,n_n,20),alpha=0.2,normed=true,log=true,label="dense")
xlim([0,100])
legend()

## Distribution of $w$

In [ ]:
figure()
bins = logspace(log10(minimum(sizes_mixed)),log10(maximum(sizes_mixed)),150)
PyPlot.plt[:hist](sizes,log=true,bins=bins,alpha=0.2,normed=true,label=L"graph, $degree = k$")
# PyPlot.plt[:hist](sizes_full,log=true,bins=bins,alpha=0.2,normed=true,label="dense")
PyPlot.plt[:hist](sizes_mixedk,log=true,bins=bins,alpha=0.2,normed=true,label=L"mixed, $degree = k$")
PyPlot.plt[:hist](sizes_mixed,log=true,bins=bins,alpha=0.2,normed=true,label=L"mixed, $degree = N-1$")


#PyPlot.plt.hist(sizes_full,log=true,bins=bins,alpha=0.5,normed=true,label="full")
gca()[:set_xscale]("log")

w_range = bins[15:end]#logspace(log10(4*minimum(sizes)),log10(maximum(sizes)),30)

P_w_th_range = normed_distribution(w_range,P_w_th(w_range,s(sqrt(w_range)./N,alpha,beta)))
P_w_th_range_eff = normed_distribution(w_range,P_w_th(w_range,get_s_eff(sqrt(w_range)./N,alpha,beta,k)))

plot(w_range,P_w_th_range,"-r",label="theory")#$P(w) \sim e^{- s(\sqrt{w})^2 w/4} w^{-3/2}/(1 + s(\sqrt{w}))$ (theory)')
plot(w_range,P_w_th_range_eff,"-g",label="effective theory")#$P(w) \sim e^{- s(\sqrt{w})^2 w/4} w^{-3/2}/(1 + s(\sqrt{w}))$ (theory)')

xlabel(L"w",size=20)
ylabel(L"P(w)",size=20)

legend(loc="lower left")
ylim([1e-8,1e1])
grid()

In [ ]:
bins

# TODO

The probability that an individual is infected
    

## Try to replicate the results of Keelings paper (using clustering coefficient) for our model

## Account for the variance in $k$ within a graph

## Analyze $N_0 P_{fix}(1/N) = P_{fix}(N_0/N)$?

## Parallelize run_epidemics

## Test whether $y$ overall is equal to $y$ as seen by the susceptibles.

## Compute $P(w)$ for small $w$ to match the experimental results (should be exponential). (Laplace transform from paper, matched asymptotic expansions)

# Random Notes

- It seems that this is because in the well mixed case we are drawing a binomial from the n population,while in the graph case we are attempting to infect the few non-infecteds. We can't get an infectivity greater than all neighbors being infected. But there will always be "deaths" in which an infected becomes uninfected.
- Formulation in terms of random birth + random death (Moran process) vs. formulation in terms of birth and death of type B's (binomial process) vs. formulation in terms of infection spread (graph process).
    
Graph process:
$$b.r. = \sum_{i \in S} n_i/k (1 + \alpha y_i) \sim N(1-y) y (1 + \alpha y)$$
Binomial process:
$$b.r. = N y (1 + \alpha y)$$



In the derivation of $y_{eff}$, how do we deal with $N \to N(1-y)$ in the sum? -> That $y$ is the $y$ of the overall graph, it just carries through the derivation.

How do we plot probability distribution over time? Can bin/smooth in both position or time space.



Tunable constraint model:
- not necessarily constant population size

Conditional models:
- some models can diverge, condition on them staying well-behaved

In [ ]:
close("all")

## Parallelization

In [1]:
#addprocs(2)
using SIS,IM,PayloadGraph,PyPlot,Epidemics
nprocs()

1

In [ ]:
@everywhere using IM

In [2]:
addprocs(2)
nprocs()
@everywhere using SIS, IM, PayloadGraph,Epidemics

In [5]:
num_runs = num_trials
ss = @spawn run_epidemic_graph(N,k,im,regular,fixation_threshold)
fetch(ss)

LoadError: LoadError: UndefVarError: num_trials not defined
while loading In[5], in expression starting on line 1

In [6]:
@everywhere begin

num_trials = 10
fixation_threshold = 2*y_n
regular=true
    
end

srand(1)
@time sizes,num_fixed,_,runs = 
    run_epidemics(N,num_trials,im, (N,im)
    -> run_epidemic_graph(N,k,im,regular,fixation_threshold));

@show sum(sizes)

srand(1)
@time sizes,num_fixed,_,runs = 
run_epidemics_parallel(N,num_trials,im, (N,im)
    -> run_epidemic_graph(N,k,im,regular,fixation_threshold));

@show sum(sizes)

LoadError: LoadError: UndefVarError: create_graph_from_value not defined
while loading In[6], in expression starting on line 155

## Speed Scaling

In [ ]:
function cat_parallel(N)
    @parallel (vcat) for i = 1:N
        [1]
    end
end

function cat_serial(N)
    return reduce(vcat)
    cat::Array{Int} = []
    for i = 1:N
        cat = vcat(cat,[1])
    end
    cat
end

function pm(N)
    return pmap(_ -> sum(randn(1000,1000)^2),1:N)
end

function sm(N)
    return map(_ -> sum(randn(1000,1000)^2),1:N)
end

In [ ]:
N = 10
@time pm(N);
@time sm(N);
#sp == ss
#(sp - ss)/ss

In [ ]:
num_trials = 100
#sizes_full,num_fixed_full,_,runs_full = run_epidemics(N,N-1,num_trials,im)

#@time run_epidemics(N,k,num_trials,im);

xx = [10,20,40,80,160]
yy = [0.035,0.085,0.31,1.01,2.8]
#yy = [0.03,0.08,0.3,1.0,3.3] #for k = 4 = const
#yy = [0.06,0.3,1.7,12] for fully connected

loglog(xx,yy)
loglog(xx,0.0005*xx.^1.7)
xlim([4,400])
grid()

#x^2.5 for fully connected graph
#x^1.7 for constant connected graph k = 4
#x^1.6 for constant connected graph k = 8
log(yy[end]/yy[1])/log(xx[end]/xx[1])